Faced alot of issue with the Encoded polylines. These polylines will contain "\\" and Python treat it them escape character, which mean when I try to save it in dataframe or in any form all the "\\" will get replaced with "\". This is a huge problem, becuase this changes the entire meaning and decoded coordinates of the encoded string. The encoded geometry should be as it is received from the API otherwise the further analysis will not be accurate. This Geometry will act as input for other APIs

After alot of hit and trials, finally solved it using the "json.dump()" method, which dump the string in varible as it is. 

In [5]:
# This is the example
new_string = "@g\\nSg\\nSce@tXgI|Ewe@dYyg@~Zyg@~ZkQvKoOnIkb@zVwSxMsZnQsh@`\\sh@b\\wBvAom@d"

# The saved string and print string a different
print(new_string)

@g\nSg\nSce@tXgI|Ewe@dYyg@~Zyg@~ZkQvKoOnIkb@zVwSxMsZnQsh@`\sh@b\wBvAom@d


Another way to solve it is using the raw string, so tried this approach as well. But this did not work with saving the values of JSON received from API, "\\" were still getting replaced with "\" when reading the reponse from the API request.

In [ ]:
new_string = r"@g\\nSg\\nSce@tXgI|Ewe@dYyg@~Zyg@~ZkQvKoOnIkb@zVwSxMsZnQsh@`\\sh@b\\wBvAom@d" 
print(new_string)

# Assuming routing_API_data['routes'][0]['geometry'] contains "\\" characters
geometry_raw = r"{}".format(routing_API_data['routes'][0]['geometry'])

# Assigning the raw string representation to the DataFrame
input_from_client.loc[index, 'New_Driving_Geometry'] = geometry_raw

Finally this is the approach that worked

In [1]:
import json
import requests
import pandas as pd

# URL of the Route API call in driving profile
routing_API_Driving_url = 'https://apis.mapmyindia.com/advancedmaps/v1/3q7gfp72nrkhdjvi19p43hp78f5k36vh/route_adv/driving/'

# Define the parameters for the API request
params = {
    'geometries': 'polyline6',
    'overview': 'full'
}

coordinates = f"72.0223,23.495345;73.819012,18.592923?"


routing_API_response = requests.get(routing_API_Driving_url + coordinates, params=params)

routing_API_data = routing_API_response.json()

# # This file to save the entire  response from the API into a text file
# filename = f"Output_route.txt"
# with open(filename, 'w', encoding='utf-8') as text_file:
#     text_file.write(json.dumps(routing_API_data, ensure_ascii=False))


df = pd.DataFrame(['Geometry_save'])

df.loc[0,'Geometry_save'] = json.dumps(routing_API_data['routes'][0]['geometry'], ensure_ascii=False).replace('"', '')

#This file is saved to check the output of this trial
filename = f"route_Geometry_test.txt"
with open(filename, 'w', encoding='utf-8') as text_file:
    text_file.write(df.loc[0,'Geometry_save'])

Now we will try to add and encoded polyline to the Map

In [7]:
from folium import Map
from folium.plugins import PolyLineFromEncoded

m = Map()

polyline_file = "GeometryCreatedwithRouteAPI/DrivingGeometry/1_drivingRoute.txt"

# with open(polyline_file, 'r') as f:
#     encoded = f.read().strip()

# print(encoded)

encoded = r"ymdkWgqstrCtEbDwAhC{I_G{d@kZyd@mZ{d@kZyd@mZ{d@kZg`@oUe`@mUqg@kZq`@sTeAe@kA[gB@_Cx@g\\nSg\\nSce@tXgI|Ewe@dYyg@~Zyg@~ZkQvKoOnIkb@zVwSxMsZnQsh@`\\sh@b\\wBvAom@d_@_MrHeg@rZeg@tZue@hYue@hYue@hY{MjGwGtBaMpCiNfBgUlAqq@x@wb@r@{d@bAyd@dA}"
PolyLineFromEncoded(encoded=encoded, color="green").add_to(m)

display(m)